# Fazendo uma nova DS
Pelo add data, consigo adicionar a competição rsna-intracranial-hemorrhage-detection ao notebook e acessar os dados sem precisar baixar!

## Imports e Instalações

In [ ]:
!mkdir ~/.kaggle
!cp ../input/kagglejson/kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import pandas as pd
import kaggle
import shutil

In [ ]:
input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection"
train_images_dir = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/"
#!ls {input_path}

### Criando os diretorios

In [ ]:
!mkdir small_rsna_ds
!mkdir small_rsna_ds/yes
!mkdir small_rsna_ds/no

## Selecionando os arquivos
*   https://www.kaggle.com/code/akensert/rsna-inceptionv3-keras-tf1-14-0?scriptVersionId=39570832 secção 4



### Definições

In [ ]:
def read_trainset(filename=input_path+"/stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        56346, 56347, 56348, 56349,
        56350, 56351, 1171830, 1171831,
        1171832, 1171833, 1171834, 1171835,
        3705312, 3705313, 3705314, 3705315,
        3705316, 3705317, 3842478, 3842479,
        3842480, 3842481, 3842482, 3842483
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

df = read_trainset()

In [ ]:
df

### Selecionando aleatoriamente

In [ ]:
df_shuffled = df.sample(frac=1)
df_shuffled

In [ ]:
df_shorten = df_shuffled[:35000]
df_shorten

## Criando o novo Dataset

In [ ]:
n = 0
while n < 34999:
    id = df_shorten.index[n]
    hemorrhage = df.loc[[id]].iat[0,0]
    
    src = train_images_dir+id+'.dcm'
    
    if(hemorrhage == 1):
        dst = './small_rsna_ds/yes/'
    else:
        dst = './small_rsna_ds/no/'
    
    shutil.copy(src, dst)
    n = n+1

In [ ]:
#!zip -rm ds_35000.zip ./small_rsna_ds